In [1]:
%matplotlib notebook
import os

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from utils.mri_viewer import MRIViewer
from utils.data_utils import im_path_to_arr
from utils.config import Config
from utils.dice_score import dice_score
from utils.data_utils import resize_data_to_original_size
from models.fcn_concat import FCN_Concat
from models.fcn_concat_v2 import FCN_Concat_v2

In [2]:
config_path = 'config_files/fcn_train_concat_2017_ablation_v3.cfg'
config = Config(config_path)
# model = FCN_Concat(config)
model = FCN_Concat_v2(config)

# for BraTS
ex_path = '/local-scratch/romain_scratch/brats2017/train/'
patient = 'Brats17_2013_10_1'
patient_path = os.path.join(ex_path, patient)
label_path = os.path.join(patient_path, patient + '_seg.nii')
label = im_path_to_arr(label_path)
flair_path = os.path.join(patient_path, patient + '_flair.nii')
flair = im_path_to_arr(flair_path)
t1post_path = os.path.join(patient_path, patient + '_t1ce.nii')
t1post = im_path_to_arr(t1post_path)

# for Rembrandt
ex_path = '/local-scratch/romain_scratch/rembrandt/train/'
patient = 'HF1398='
patient_path = os.path.join(ex_path, patient)
label_path = os.path.join(patient_path, 'seg.nii')
label = im_path_to_arr(label_path)
flair_path = os.path.join(patient_path, 'flair.nii')
flair = im_path_to_arr(flair_path)
t1post_path = os.path.join(patient_path, 't1c.nii')
t1post = im_path_to_arr(t1post_path)

### select a patient id between 1 and 193

In [3]:
patient_id = 10

In [4]:
# for RTOG
ex_path = '/local-scratch/romain_scratch/rtog/'
patient = os.listdir(ex_path)[patient_id - 1]
patient_path = os.path.join(ex_path, patient)
flair_path = os.path.join(patient_path, 'flair.nii')
flair = im_path_to_arr(flair_path)
t1post_path = os.path.join(patient_path, 't1c.nii')
t1post = im_path_to_arr(t1post_path)
print('shape of this patient is:', t1post.shape)

shape of this patient is: (30, 512, 512)


In [4]:
import gzip

def j(path, fname):
    return os.path.join(path, fname)

patient_path = '/home/romains/rtog_ex2/post'

with gzip.open(j(patient_path, 'flair.nii.gz'), 'rb') as fgz:
            with open(j(patient_path, 'flair.nii'), 'wb') as f:
                f.write(fgz.read())
                
with gzip.open(j(patient_path, 't1c.nii.gz'), 'rb') as fgz:
            with open(j(patient_path, 't1c.nii'), 'wb') as f:
                f.write(fgz.read())

t1post = im_path_to_arr(j(patient_path, 't1c.nii'))
flair = im_path_to_arr(j(patient_path, 'flair.nii'))

patient_path = '/home/romains/rtog_patient_post_2'

with gzip.open(j(patient_path, 'flair_bcorr_brain.nii.gz'), 'rb') as fgz:
            with open(j(patient_path, 'flair.nii'), 'wb') as f:
                f.write(fgz.read())
                
with gzip.open(j(patient_path, 't1_bcorr_brain.nii.gz'), 'rb') as fgz:
            with open(j(patient_path, 't1c.nii'), 'wb') as f:
                f.write(fgz.read())

t1post = im_path_to_arr(j(patient_path, 't1c.nii'))
flair = im_path_to_arr(j(patient_path, 'flair.nii'))

In [5]:
patient_path = patient_path.encode('utf-8')
saver = tf.train.Saver()
conf = tf.ConfigProto()
conf.gpu_options.allow_growth = True
with tf.Session() as sess:
    saver.restore(sess, config.ckpt_path)
    pred = model.run_pred_single_example_v3(sess, patient_path)
    pred[pred == 3] = 4
patient_path = patient_path.decode('utf-8')

INFO:tensorflow:Restoring parameters from ckpt/fcn_brats_concat_2017_ablation_v3.ckpt


InternalError: Dst tensor is not initialized.
	 [[Node: IteratorGetNext/_301 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_6_IteratorGetNext", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

In [ ]:
# for Rembrandt and RTOG
# resize to original size
original_size = im_path_to_arr(os.path.join(patient_path, 'flair.nii')).shape
pred = resize_data_to_original_size(pred, original_size)

# for Brats and Rembrandt
def get_dice_scores(p, y):
    dice_whole = dice_score(y, pred)
    fpred_core = (p == 1) + (p == 4)
    fy_core = (y == 1) + (y == 4)
    dice_core = dice_score(fy_core, fpred_core)
    fpred_enhancing = p == 4
    fy_enhancing = y == 4
    dice_enhancing = dice_score(fy_enhancing, fpred_enhancing)
    print('whole: %f ... core: %f ... enhancing: %f ...'%(dice_whole, dice_core, dice_enhancing))
get_dice_scores(pred, label)

# for Brats and Rembrandt
flair_pred = np.array(flair)
flair_pred[pred == 1] = 1000
flair_pred[pred == 2] = 2000
flair_pred[pred == 4] = 4000
flair_label = np.array(flair)
flair_label[label == 1] = 1000
flair_label[label == 2] = 2000
flair_label[label == 4] = 4000
MRIViewer(flair_pred, flair_label, flair, t1post, cmap=None, title1='prediction', title2='ground truth', title3='flair', title4='t1post')

In [ ]:
# for RTOG
flair_pred = np.array(flair)
flair_pred[pred == 1] = 1000
flair_pred[pred == 2] = 2000
flair_pred[pred == 4] = 4000
MRIViewer(flair_pred, flair_pred, flair, t1post, cmap=None, title1='prediction', title2='prediction', title3='flair', title4='t1post')

In [ ]:
patient_path